In [241]:
import torch
import torch.nn as nn
from models.MIL_VT import MILVisionTransformer
from models.FinetuneVTmodels import MIL_VT_FineTune

In [ ]:
print(dir(model))

MIL-VT Pretrained 

In [2]:
from models.MIL_VT import MIL_VT_small_patch16_384
model =MIL_VT_small_patch16_384()
model
#model.state_dict().keys()

MILVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (l

In [45]:
model.eval()
embedding_dim = model.patch_embed.proj.out_channels
print("Embedding Dimension:", embedding_dim)

Embedding Dimension: 384


In [242]:
weight_path_tar ='/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/weights/fundus_pretrained_VT_small_patch16_384_5Class.pth.tar'

In [ ]:
weight_path ='/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/weights/archieve/data'
import tarfile
with tarfile.open(weight_path_tar,"r") as tar:
    tar.extractall(weight_path)

In [46]:
# Update the model's state_dict
#model_dict.update(checkpoint)

# Load the updated state_dict into the model
#model.load_state_dict(model_dict)
#model.load_state_dict()
import torch
checkpoint =torch.load(weight_path_tar)
checkpoint['state_dict'].keys()

odict_keys(['module.cls_token', 'module.pos_embed', 'module.dist_token', 'module.patch_embed.proj.weight', 'module.patch_embed.proj.bias', 'module.blocks.0.norm1.weight', 'module.blocks.0.norm1.bias', 'module.blocks.0.attn.qkv.weight', 'module.blocks.0.attn.qkv.bias', 'module.blocks.0.attn.proj.weight', 'module.blocks.0.attn.proj.bias', 'module.blocks.0.norm2.weight', 'module.blocks.0.norm2.bias', 'module.blocks.0.mlp.fc1.weight', 'module.blocks.0.mlp.fc1.bias', 'module.blocks.0.mlp.fc2.weight', 'module.blocks.0.mlp.fc2.bias', 'module.blocks.1.norm1.weight', 'module.blocks.1.norm1.bias', 'module.blocks.1.attn.qkv.weight', 'module.blocks.1.attn.qkv.bias', 'module.blocks.1.attn.proj.weight', 'module.blocks.1.attn.proj.bias', 'module.blocks.1.norm2.weight', 'module.blocks.1.norm2.bias', 'module.blocks.1.mlp.fc1.weight', 'module.blocks.1.mlp.fc1.bias', 'module.blocks.1.mlp.fc2.weight', 'module.blocks.1.mlp.fc2.bias', 'module.blocks.2.norm1.weight', 'module.blocks.2.norm1.bias', 'module.blo

In [36]:
from PIL import Image
from torchvision import transforms

image_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/MultiClass/train/image10.jpg'
image = Image.open(image_path).convert('RGB')

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Apply the transformation
input_image = transform(image).unsqueeze(0)  # Add batch dimension


In [37]:
input_image.shape

torch.Size([1, 3, 384, 384])

In [251]:
#MIL-VT output
from PIL import Image
from torchvision import transforms
import os
from transformers import ViTFeatureExtractor
import torch
from models.FinetuneVTmodels import MIL_VT_FineTune
#from models.MIL_VT import MIL_VT_small_patch16_384


# Create an instance of your custom model
model = MIL_VT_FineTune()
checkpoint = 'weights/fundus_pretrained_VT_small_patch16_384_5Class.pth.tar'

model.eval()

class_names=['normal','DR','ARMD','glaucoma','cataract']

result=[]
root_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/image_set5/'

#image_files = [f for f in os.listdir(root_path) if f.lower().endswith(('.jpeg','.png','.jpg','.JPEG'))]

transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

for img in sorted(os.listdir(root_path)):
    
    if img.lower().endswith(('.jpeg','.png','.jpg','.JPEG')):
        image_path =os.path.join(root_path,img)
        print(image_path)
        image = Image.open(image_path)
        input_image = transform(image).unsqueeze(0)
        #print(f"max value:{torch.max(input_image)}")
        #print(f"min value:{torch.min(input_image)}")

   
        with torch.no_grad():
            #embedding,_= model.forward_features(input_image)
            output=model(input_image)
        print(output)
        result.append(output)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        
        
        predicted_class = torch.argmax(probabilities, dim=1).item()
        label = class_names[predicted_class]
        
        # Append the result to the list
        #result.append({'image_path': image_path, 'predicted_class': label, 'probabilities': probabilities.tolist()})

    
      
      
        
    

Timm here
Missing Key not in pretrain model:  MIL_Prep.0.weight
Missing Key not in pretrain model:  MIL_Prep.0.bias
Missing Key not in pretrain model:  MIL_Prep.1.weight
Missing Key not in pretrain model:  MIL_Prep.1.bias
Missing Key not in pretrain model:  MIL_attention.0.weight
Missing Key not in pretrain model:  MIL_attention.0.bias
Missing Key not in pretrain model:  MIL_attention.1.weight
Missing Key not in pretrain model:  MIL_attention.1.bias
Missing Key not in pretrain model:  MIL_attention.4.weight
Missing Key not in pretrain model:  MIL_attention.4.bias
Missing Key not in pretrain model:  MIL_classifier.0.weight
Missing Key not in pretrain model:  MIL_classifier.0.bias
Removing key head.weight from pretrained checkpoint
Removing key head.bias from pretrained checkpoint
Removing key head_dist.weight from pretrained checkpoint
Removing key head_dist.bias from pretrained checkpoint
pos_embed:  384 576 1
/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical

In [252]:
#print(result)
print(probabilities)

tensor([[0.1792, 0.1342, 0.2205, 0.1752, 0.2910]])


In [105]:
#PGD
# Perform projection to ensure the updated_input is within the feasible set
#projected_input = cur_input + torch.clamp(updated_input - cur_input, -epsilon, epsilon)

[tensor([[-0.1002, -0.0783, -0.0792, -0.1269,  0.0592]]),
 tensor([[-0.2693, -0.2108,  0.2600, -0.2633,  0.3379]]),
 tensor([[-0.0609, -0.2154, -0.0686,  0.0233,  0.0599]]),
 tensor([[ 0.0025, -0.3373,  0.4450,  0.1035, -0.1666]]),
 tensor([[ 0.0822,  0.2182,  0.4722,  0.4829, -0.1891]])]

In [232]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests
image_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/image_set5/3_l3.jpeg'
image = Image.open(image_path)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-384')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-384')
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])


/home/montasir/.local/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Predicted class: nematode, nematode worm, roundworm


In [92]:
torch.min(inputs['pixel_values'])

tensor(-1.)

In [17]:
with torch.no_grad():
    logits = model(input_image).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

LABEL_1


MIL-VT-Finetune for image embeddings

In [193]:
#Splitting image into train and test set

import os
import shutil

original_folder ='/home/montasir/Downloads/APTOS/resized_train/resized_train'
training_folder = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/train'
test_folder = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test'

 #directory creation
if not os.path.exists(training_folder):
    os.makedirs(training_folder)

if not os.path.exists(test_folder):
    os.makedirs(test_folder)
    


In [194]:
all_images =sorted([f for f in os.listdir(original_folder) if f.lower().endswith(('.jpeg','.jpg','.JPEG','.png'))])
len(all_images)
num_training_images= 4000
num_test_images= 500

# Move the first 4000 images to the training folder
for img in all_images[:num_training_images]:
    src_path = os.path.join(original_folder, img)
    #print(src_path)
    dst_path = os.path.join(training_folder, img)
    shutil.copy(src_path, dst_path)



for img in all_images[5000:5500]:
    src_path = os.path.join(original_folder, img)
    print(src_path)
    dst_path = os.path.join(test_folder, img)
    shutil.copy(src_path, dst_path)
    
    





/home/montasir/Downloads/APTOS/resized_train/resized_train/15661_left.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15661_right.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15666_left.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15666_right.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15667_left.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15667_right.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15674_left.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15674_right.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15682_left.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15682_right.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15683_left.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15683_right.jpeg
/home/montasir/Downloads/APTOS/resized_train/resized_train/15688_left.jpeg
/home/montasir/Down

In [233]:
from PIL import Image
from torchvision import transforms
import os
import pandas as pd
import torch
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split


def apply_transform(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    input_image = transform(image).unsqueeze(0)
    return input_image

def generate_embeddings(model, data_root_path, label_info,save_path,device):
    embeddings = []
    labels = []
    training_image_paths=[]
    global count
    
    model.to(device)

    transform = transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    for img_name in sorted(os.listdir(data_root_path)):
        if img_name.lower().endswith(('.jpeg', '.png', '.jpg','.JPEG')):
            image_path = os.path.join(data_root_path, img_name)
            #print(image_path)
            input_image = apply_transform(image_path, transform).to(device)

            with torch.no_grad():
                output,_ = model.forward_features(input_image)
            
            matching_rows = label_info[label_info['image_path'] == image_path]
            #print(matching_rows)
            if not matching_rows.empty:
                #label = matching_rows['Classes'].values[0]
                label = matching_rows['level'].values[0]
                embeddings.append(output.squeeze().cpu().numpy())
                labels.append(label)
                training_image_paths.append(image_path)
            else:
                print(image_path)

    embeddings =np.array(embeddings)
    labels = np.array(labels)
    if data_root_path == training_path:
        torch.save({'embeddings': embeddings, 'labels': labels}, save_path)
        return embeddings, labels, training_image_paths
    
    return embeddings,labels,training_image_paths

training_directory = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/train/'
testing_directory = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test/'
training_csv_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/trainLabels.csv'
testing_csv_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/testLabels.csv'
training_label_info = pd.read_csv(training_csv_path)
test_label_info = pd.read_csv(testing_csv_path)

training_label_info['image_path'] = training_directory + training_label_info['image'] +'.jpeg'
test_label_info['image_path'] = testing_directory + test_label_info['image']  +'.jpeg'
# Load  MIL-VT-Finetune model
model = MIL_VT_FineTune()

model.eval()

# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



training_path ='/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/train'
test_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test'


train_save_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/trainData_embeddings_levels.pth'
test_save_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/testData_embeddings_levels.pth'


#Generate embeddings for the training set


#embedding for test set



Timm here
Missing Key not in pretrain model:  MIL_Prep.0.weight
Missing Key not in pretrain model:  MIL_Prep.0.bias
Missing Key not in pretrain model:  MIL_Prep.1.weight
Missing Key not in pretrain model:  MIL_Prep.1.bias
Missing Key not in pretrain model:  MIL_attention.0.weight
Missing Key not in pretrain model:  MIL_attention.0.bias
Missing Key not in pretrain model:  MIL_attention.1.weight
Missing Key not in pretrain model:  MIL_attention.1.bias
Missing Key not in pretrain model:  MIL_attention.4.weight
Missing Key not in pretrain model:  MIL_attention.4.bias
Missing Key not in pretrain model:  MIL_classifier.0.weight
Missing Key not in pretrain model:  MIL_classifier.0.bias
Removing key head.weight from pretrained checkpoint
Removing key head.bias from pretrained checkpoint
Removing key head_dist.weight from pretrained checkpoint
Removing key head_dist.bias from pretrained checkpoint
pos_embed:  384 576 1


In [196]:
training_embeddings, training_labels, training_image_paths = generate_embeddings(model, data_root_path=training_path, label_info=training_label_info, save_path=train_save_path, device=device)


In [234]:
test_embeddings, test_labels,test_image_paths = generate_embeddings(model, data_root_path=test_path, label_info=test_label_info, save_path=test_save_path, device=device)

In [235]:
len(test_labels)

500

In [236]:
nn_model =NearestNeighbors(n_neighbors=1,metric='cosine')
nn_model.fit(training_embeddings)

top_neighbors=[]

for test_sample in test_embeddings:
    distances, indices =nn_model.kneighbors([test_sample])
    #print(test_sample)
    #print(indices[0])
    top_neighbors.append(indices[0])
    

In [237]:
top_neighbors

[array([827]),
 array([605]),
 array([966]),
 array([373]),
 array([2697]),
 array([3414]),
 array([3060]),
 array([3060]),
 array([1151]),
 array([2372]),
 array([2784]),
 array([3566]),
 array([3884]),
 array([713]),
 array([2085]),
 array([2604]),
 array([1160]),
 array([1137]),
 array([1919]),
 array([3152]),
 array([2710]),
 array([357]),
 array([3542]),
 array([2239]),
 array([2221]),
 array([3913]),
 array([2854]),
 array([1549]),
 array([370]),
 array([243]),
 array([3298]),
 array([3393]),
 array([2366]),
 array([817]),
 array([2127]),
 array([2719]),
 array([220]),
 array([1701]),
 array([792]),
 array([388]),
 array([2273]),
 array([2270]),
 array([3737]),
 array([3481]),
 array([2298]),
 array([2353]),
 array([3835]),
 array([3925]),
 array([3307]),
 array([3167]),
 array([1565]),
 array([2621]),
 array([3298]),
 array([3179]),
 array([1234]),
 array([3326]),
 array([2214]),
 array([455]),
 array([1815]),
 array([1517]),
 array([3714]),
 array([1803]),
 array([2614]),
 arra

In [238]:
flattened_neighbors = []
flattened_neighbors

for neighbors_list in top_neighbors:
    #print(neighbors_list)
    for neighbor in neighbors_list:
        flattened_neighbors.append(neighbor)

In [240]:
flattened_neighbors = []
flattened_neighbors

for neighbors_list in top_neighbors:
    #print(neighbors_list)
    for neighbor in neighbors_list:
        flattened_neighbors.append(neighbor)

from sklearn.metrics import accuracy_score
predicted_labels = training_labels[flattened_neighbors]


# Calculate accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print(predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 1 0 3 1 0 0 0 0 0 2 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
 4 4 2 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 3 2 0 0 0 0 3 0 0
 0 2 0 0 0 2 0 2 0 0 0 0 0 0 0 0 0 1 0 2 0 0 0 0 0 2 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 2 0 2 2 1 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0
 0 2 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 4 2 0 0 2 2 0 0 0 2 0 0 0 2 0 0 0 0 2 0
 2 0 0 4 1 1 0 0 0 0 3 2 0 0 0 0 2 2 0 1 2 2 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0
 0 3 3 0 0 0 0 2 0 0 0 0 1 2 0 3 3 0 0 0 0 0 0 0 0 2 1 0 0 0 2 2 2 2 2 0 0
 3 2 0 1 2 2 2 3 0 2 0 0 0 1 0 2 0 0 2 2 0 4 1 0 0 0 2 2 4 2 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 2 2 0 0 2 0 0 0 3 0 1 0 2 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 2 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
 2 0 1 1 0 0 2 0 2 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 2 2 2 0 0 0 0 0 0 2 2 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 2 2 0 2 2 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 0 0 0 0 0 0 0 0 0 

In [172]:
test_labels

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [4],
       [4],
       [0],
       [0],
       [1],
       [0],
       [3],
       [3],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [2],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [2],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [2],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [2],
       [3],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [4],
    

In [75]:
training_label_info[' Classes'][1]

'  sitar'

In [62]:
# Display or save the results as needed
for i, neighbors in enumerate(top_3_neighbors):
    # Extract the numeric part of the test image filename
    test_image_number = ''.join(filter(str.isdigit, test_image_paths[i]))

    # Extract the numeric parts of the top-5 neighbors' filenames
    training_image_numbers = [''.join(filter(str.isdigit, training_image_paths[idx])) for idx in neighbors]

    print(f"Test image {test_image_number}: Top-3 neighbors:", training_image_numbers)


Test image 2024761: Top-3 neighbors: ['2024360', '20244215', '20241105']
Test image 2024762: Top-3 neighbors: ['20241167', '20241323', '20241059']
Test image 2024763: Top-3 neighbors: ['20241208', '20241048', '2024304']
Test image 2024764: Top-3 neighbors: ['20244264', '20243328', '20243256']
Test image 2024765: Top-3 neighbors: ['20242521', '20242676', '20241274']
Test image 2024766: Top-3 neighbors: ['20244190', '20242166', '20242291']
Test image 2024767: Top-3 neighbors: ['20242344', '20242769', '20242699']
Test image 2024768: Top-3 neighbors: ['20244262', '20244045', '20244193']
Test image 2024769: Top-3 neighbors: ['20241061', '20241302', '20241274']
Test image 202477: Top-3 neighbors: ['20241604', '2024114', '20241641']
Test image 2024770: Top-3 neighbors: ['20241318', '20242422', '20242562']
Test image 2024771: Top-3 neighbors: ['20242488', '20241236', '2024238']
Test image 2024772: Top-3 neighbors: ['20244159', '20241184', '20241197']
Test image 2024773: Top-3 neighbors: ['2024

In [54]:
import pandas as pd
testing_csv_path_csv = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/MultiClass/multilabel_classification_train.csv'
test_label_info = pd.read_csv(testing_csv_path,skiprows=1)
'''
# Assuming your images are in a directory called 'images'
image_directory = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/MultiClass/train/'

# Add a new column 'image_path' to the DataFrame with the full path
training_label_info['image_path'] = image_directory + training_label_info['image'] + '.jpeg'
training_label_info['image_path'][7535]
'''
test_label_info['image_path'] = testing_directory + test_label_info['Image_Name'] 








Testing with modified images

In [225]:
from PIL import Image
from torchvision import transforms
import os
import pandas as pd
import torch
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split


def apply_transform(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    input_image = transform(image).unsqueeze(0)
    return input_image

def generate_embeddings(model, data_root_path, label_info,save_path,device):
    embeddings = []
    labels = []
    training_image_paths=[]
    global count
    
    model.to(device)

    transform = transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    for img_name in sorted(os.listdir(data_root_path)):
        if img_name.lower().endswith(('.jpeg', '.png', '.jpg','.JPEG')):
            image_path = os.path.join(data_root_path, img_name)
            print(image_path)
            input_image = apply_transform(image_path, transform).to(device)

            with torch.no_grad():
                output,_ = model.forward_features(input_image)
            
            matching_rows = label_info[label_info['image_path'] == image_path]
            #print(matching_rows)
            if not matching_rows.empty:
                #label = matching_rows['Classes'].values[0]
                label = matching_rows['level'].values[0]
                embeddings.append(output.squeeze().cpu().numpy())
                labels.append(label)
                #training_image_paths.append(image_path)
            else:
                print(image_path)

    embeddings =np.array(embeddings)
    labels = np.array(labels)
    if data_root_path == training_path:
        torch.save({'embeddings': embeddings, 'labels': labels}, save_path)
        return embeddings, labels
    
    return embeddings,labels

training_directory = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/train/'
#testing_directory = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test/'
testing_directory = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo/'

training_csv_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/trainLabels.csv'
#testing_csv_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/testLabels.csv'
testing_csv_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo.csv'
training_label_info = pd.read_csv(training_csv_path)
test_label_info = pd.read_csv(testing_csv_path)

training_label_info['image_path'] = training_directory + training_label_info['image'] 
test_label_info['image_path'] = testing_directory + test_label_info['image'] 
# Load  MIL-VT-Finetune model
model = MIL_VT_FineTune()

model.eval()

# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



training_path ='/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/train'
test_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo'


train_save_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/trainData_embeddings_levels.pth'
test_save_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/testData_embeddings_levels.pth'


#Generate embeddings for the training set


#embedding for test set



Timm here
Missing Key not in pretrain model:  MIL_Prep.0.weight
Missing Key not in pretrain model:  MIL_Prep.0.bias
Missing Key not in pretrain model:  MIL_Prep.1.weight
Missing Key not in pretrain model:  MIL_Prep.1.bias
Missing Key not in pretrain model:  MIL_attention.0.weight
Missing Key not in pretrain model:  MIL_attention.0.bias
Missing Key not in pretrain model:  MIL_attention.1.weight
Missing Key not in pretrain model:  MIL_attention.1.bias
Missing Key not in pretrain model:  MIL_attention.4.weight
Missing Key not in pretrain model:  MIL_attention.4.bias
Missing Key not in pretrain model:  MIL_classifier.0.weight
Missing Key not in pretrain model:  MIL_classifier.0.bias
Removing key head.weight from pretrained checkpoint
Removing key head.bias from pretrained checkpoint
Removing key head_dist.weight from pretrained checkpoint
Removing key head_dist.bias from pretrained checkpoint
pos_embed:  384 576 1


In [226]:
test_embeddings, test_labels = generate_embeddings(model, data_root_path=test_path, label_info=test_label_info, save_path=test_save_path, device=device)

/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo/0_l0.jpeg
/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo/1_l1.jpeg
/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo/2_l2.jpeg
/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo/3_l3.jpeg
/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo/4_l4.jpeg
/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo/l0>l4.png
/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/APTOS2019/test_demo/l4>l0.png
/home/montasir/Desktop/Montasir/Spring 2024/Research and Exper

In [227]:
test_labels

array([0, 1, 2, 3, 4, 4, 0, 1, 2, 3])

In [228]:
nn_model =NearestNeighbors(n_neighbors=1,metric='cosine')
nn_model.fit(training_embeddings)

top_neighbors=[]

for test_sample in test_embeddings:
    distances, indices =nn_model.kneighbors([test_sample])
    #print(test_sample)
    #print(indices[0])
    top_neighbors.append(indices[0])
    

In [229]:
top_neighbors

[array([0]),
 array([64]),
 array([110]),
 array([96]),
 array([16]),
 array([16]),
 array([1189]),
 array([64]),
 array([3731]),
 array([96])]

In [230]:

from sklearn.metrics import accuracy_score
flattened_neighbors = []
flattened_neighbors

for neighbors_list in top_neighbors:
    #print(neighbors_list)
    for neighbor in neighbors_list:
        flattened_neighbors.append(neighbor)

predicted_labels = training_labels[flattened_neighbors]


# Calculate accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print(predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

[0 1 2 3 4 4 0 1 0 3]
Accuracy: 90.00%
